In [1]:
# Dependencies
import os 
import csv 
import pandas as pd 
import numpy as np

In [2]:
# load CSV
schools_csv="Resources/schools_complete.csv"
students_csv="Resources/students_complete.csv"

In [19]:
# Read with pandas
schools_original_pd = pd.read_csv(schools_csv)
students_original_pd = pd.read_csv(students_csv)
schools_df = schools_original_pd.rename(columns={"name":"School", "type": "Type","size":"SchoolSize","budget": "TotalBudget"})
students_df = students_original_pd.rename(columns={"name":"StudentName", "gender": "Gender", "school":"School", 
                                                    "grade": "Grade", "reading_score": "ReadingScore","math_score": "MathScore"}) 
                                                    
                                               
students_df.count()

Student ID      39170
StudentName     39170
Gender          39170
Grade           39170
School          39170
ReadingScore    39170
MathScore       39170
dtype: int64

In [20]:
students_df.head()

,Student ID,StudentName,Gender,Grade,School,ReadingScore,MathScore
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [52]:
schools_df.head()

,School ID,School,Type,SchoolSize,TotalBudget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [21]:
# merge data frames
academy_df=pd.merge(students_df,schools_df, on="School", how="outer")
academy_df.head()

,Student ID,StudentName,Gender,Grade,School,ReadingScore,MathScore,School ID,Type,SchoolSize,TotalBudget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [224]:
academy_df.groupby(['Grade']).count()

,Student ID,StudentName,Gender,School,ReadingScore,MathScore,School ID,Type,SchoolSize,TotalBudget,Budget per Student
Grade,,,,,,,,,,,
10th,10168,10168,10168,10168,10168,10168,10168,10168,10168,10168,10168
11th,9695,9695,9695,9695,9695,9695,9695,9695,9695,9695,9695
12th,7899,7899,7899,7899,7899,7899,7899,7899,7899,7899,7899
9th,11408,11408,11408,11408,11408,11408,11408,11408,11408,11408,11408


# Disrict Summary

In [95]:
Total_Schools = academy_df.groupby(["School"])["Type"].unique().value_counts().sum()
Total_students = academy_df["StudentName"].value_counts().sum()
Total_budget = academy_df.groupby(["School"])["TotalBudget"].mean().sum()
Average_math_score = academy_df.MathScore.mean()
Average_reading_score = academy_df.ReadingScore.mean()
Total_Reading_passing = academy_df.loc[academy_df["ReadingScore"] >=70,:]["StudentName"].count()
Reading_passing_pct = Total_Reading_passing/Total_students *100
Total_math_passing = academy_df.loc[academy_df["MathScore"] >=70,:]["StudentName"].count()
Math_passing_pct = Total_math_passing/Total_students *100
Overall_passing = (Reading_passing_pct+Math_passing_pct)/2
Overall_passing

80.393158029103915

In [99]:
district_summary = pd.DataFrame({"Total Schools": [Total_Schools],"Total Budget": Total_budget, "Average Math Score": Average_math_score,
                                 "Average Reading Score": Average_reading_score, "Passing Math %": Math_passing_pct, 
                                 "Passing Reading %": Reading_passing_pct, "Overall Passing %": Overall_passing})
district_summary

,Average Math Score,Average Reading Score,Overall Passing %,Passing Math %,Passing Reading %,Total Budget,Total Schools
0,78.985371,81.87784,80.393158,74.980853,85.805463,24649428,15


# School Summary

In [160]:
Unique_Schools = academy_df.School.unique()
School_type = academy_df.groupby(["School"])["Type"].unique()
Total_students_perschool = academy_df["School"].value_counts()
Total_budget_perschool = academy_df.groupby(["School"])["TotalBudget"].mean()
Budget_per_student = Total_budget_perschool/Total_students_perschool
Average_math_perschool = academy_df.groupby(["School"])["MathScore"].mean()
Average_reading_perschool = academy_df.groupby(["School"])["ReadingScore"].mean()
Reading_passing_perschool = academy_df.loc[academy_df["ReadingScore"] >=70,"School"].value_counts()
Math_passing_perschool = academy_df.loc[academy_df["MathScore"] >=70,"School"].value_counts()
Reading_passing_pct_perschool = Reading_passing_perschool/Total_students_perschool *100
Math_passing_pct_perschool= Math_passing_perschool/Total_students_perschool *100
Overall_passing_perschool = (Reading_passing_pct_perschool+Math_passing_pct_perschool)/2
Unique_Schools

array(['Huang High School', 'Figueroa High School', 'Shelton High School',
       'Hernandez High School', 'Griffin High School',
       'Wilson High School', 'Cabrera High School', 'Bailey High School',
       'Holden High School', 'Pena High School', 'Wright High School',
       'Rodriguez High School', 'Johnson High School', 'Ford High School',
       'Thomas High School'], dtype=object)

In [188]:
school_summary_perschool = pd.DataFrame({"School Type": School_type, 
                            "Total Students": Total_students_perschool,
                            "Total Budget": Total_budget_perschool,
                            "Budget per Student": Budget_per_student,
                            "Average Math Score %": Average_math_perschool, 
                            "Average Reading Score %": Average_reading_perschool,
                            "Passing Reading %": Reading_passing_pct_perschool,
                            "Passing Math %": Math_passing_pct_perschool,
                            "Overall Passing %": Overall_passing_perschool
                                        })

cols = ['School Type','Total Students','Total Budget','Budget per Student', 'Average Math Score %', 
        'Passing Math %','Average Reading Score %', 'Passing Reading %', 'Overall Passing %']
Schools_summary_final = school_summary_perschool[cols]
Schools_summary_final

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score %,Passing Math %,Average Reading Score %,Passing Reading %,Overall Passing %
Bailey High School,[District],4976,3124928,628.0,77.048432,66.680064,81.033963,81.933280,74.306672
Cabrera High School,[Charter],1858,1081356,582.0,83.061895,94.133477,83.975780,97.039828,95.586652
Figueroa High School,[District],2949,1884411,639.0,76.711767,65.988471,81.158020,80.739234,73.363852
Ford High School,[District],2739,1763916,644.0,77.102592,68.309602,80.746258,79.299014,73.804308
Griffin High School,[Charter],1468,917500,625.0,83.351499,93.392371,83.816757,97.138965,95.265668
Hernandez High School,[District],4635,3022020,652.0,77.289752,66.752967,80.934412,80.862999,73.807983
Holden High School,[Charter],427,248087,581.0,83.803279,92.505855,83.814988,96.252927,94.379391
Huang High School,[District],2917,1910635,655.0,76.629414,65.683922,81.182722,81.316421,73.500171
Johnson High School,[District],4761,3094650,650.0,77.072464,66.057551,80.966394,81.222432,73.639992
Pena High School,[Charter],962,585858,609.0,83.839917,94.594595,84.044699,95.945946,95.270270


# Top 5 schools

In [220]:
Top_performing_schools = Schools_summary_final.sort_values('Overall Passing %',ascending=False).head(5)
Top_performing_schools

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score %,Passing Math %,Average Reading Score %,Passing Reading %,Overall Passing %,Budget Bin,School Size
Cabrera High School,['Charter'],1858,1081356,582.0,83.061895,94.133477,83.975780,97.039828,95.586652,<$600,Small
Thomas High School,['Charter'],1635,1043130,638.0,83.418349,93.272171,83.848930,97.308869,95.290520,$621-640,Small
Pena High School,['Charter'],962,585858,609.0,83.839917,94.594595,84.044699,95.945946,95.270270,$601-620,Small
Griffin High School,['Charter'],1468,917500,625.0,83.351499,93.392371,83.816757,97.138965,95.265668,$621-640,Small
Wilson High School,['Charter'],2283,1319574,578.0,83.274201,93.867718,83.989488,96.539641,95.203679,<$600,Medium


# Bottom 5 Schools

In [190]:
Bottom_performing_schools = Schools_summary_final.sort_values('Overall Passing %',ascending=True).head(5)
Bottom_performing_schools

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score %,Passing Math %,Average Reading Score %,Passing Reading %,Overall Passing %
Rodriguez High School,[District],3999,2547363,637.0,76.842711,66.366592,80.744686,80.220055,73.293323
Figueroa High School,[District],2949,1884411,639.0,76.711767,65.988471,81.158020,80.739234,73.363852
Huang High School,[District],2917,1910635,655.0,76.629414,65.683922,81.182722,81.316421,73.500171
Johnson High School,[District],4761,3094650,650.0,77.072464,66.057551,80.966394,81.222432,73.639992
Ford High School,[District],2739,1763916,644.0,77.102592,68.309602,80.746258,79.299014,73.804308


# Average Reading Scores by grade

In [191]:
grade9=academy_df.loc[academy_df["Grade"]=="9th",:]
grade10=academy_df.loc[academy_df["Grade"]=="10th",:]
grade11=academy_df.loc[academy_df["Grade"]=="11th",:]
grade12=academy_df.loc[academy_df["Grade"]=="12th",:]

In [192]:
grade9_reading_score = grade9.groupby("School")["ReadingScore"].mean()
grade10_reading_score = grade10.groupby("School")["ReadingScore"].mean()
grade11_reading_score = grade11.groupby("School")["ReadingScore"].mean()
grade12_reading_score = grade12.groupby("School")["ReadingScore"].mean()

Reading_by_grade = pd.DataFrame({'9th Grade Average': grade9_reading_score,'10th Grade Average': grade10_reading_score, 
                                 '11th Grade Average': grade11_reading_score, '12th Grade Average': grade12_reading_score})

Reading_by_grade

,10th Grade Average,11th Grade Average,12th Grade Average,9th Grade Average
School,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


# Average Math Scores by Grade

In [193]:
grade9_math_score = grade9.groupby("School")["MathScore"].mean()
grade10_math_score = grade10.groupby("School")["MathScore"].mean()
grade11_math_score = grade11.groupby("School")["MathScore"].mean()
grade12_math_score = grade12.groupby("School")["MathScore"].mean()

Math_by_grade = pd.DataFrame({'9th Grade Average': grade9_math_score,'10th Grade Average': grade10_math_score, 
                                 '11th Grade Average': grade11_math_score, '12th Grade Average': grade12_math_score})

Reading_by_grade

,10th Grade Average,11th Grade Average,12th Grade Average,9th Grade Average
School,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


# Scores by School Spending

In [198]:
academy_adjusted_df=pd.DataFrame(Schools_summary_final)
budget_bins = [0,600,620,640,1000]
budget_bin_labels = ['<$600','$601-620','$621-640','>$641']
academy_adjusted_df["Budget Bin"]= pd.cut(Schools_summary_final["Budget per Student"],budget_bins,labels=budget_bin_labels)
size_bins =[0,2000,4000,5000]
size_bin_labels = ['Small','Medium','Large']
academy_adjusted_df["School Size"]= pd.cut(Schools_summary_final["Total Students"],size_bins,labels=size_bin_labels)
academy_adjusted_df

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score %,Passing Math %,Average Reading Score %,Passing Reading %,Overall Passing %,Budget Bin,School Size
Bailey High School,[District],4976,3124928,628.0,77.048432,66.680064,81.033963,81.933280,74.306672,$621-640,Large
Cabrera High School,[Charter],1858,1081356,582.0,83.061895,94.133477,83.975780,97.039828,95.586652,<$600,Small
Figueroa High School,[District],2949,1884411,639.0,76.711767,65.988471,81.158020,80.739234,73.363852,$621-640,Medium
Ford High School,[District],2739,1763916,644.0,77.102592,68.309602,80.746258,79.299014,73.804308,>$641,Medium
Griffin High School,[Charter],1468,917500,625.0,83.351499,93.392371,83.816757,97.138965,95.265668,$621-640,Small
Hernandez High School,[District],4635,3022020,652.0,77.289752,66.752967,80.934412,80.862999,73.807983,>$641,Large
Holden High School,[Charter],427,248087,581.0,83.803279,92.505855,83.814988,96.252927,94.379391,<$600,Small
Huang High School,[District],2917,1910635,655.0,76.629414,65.683922,81.182722,81.316421,73.500171,>$641,Medium
Johnson High School,[District],4761,3094650,650.0,77.072464,66.057551,80.966394,81.222432,73.639992,>$641,Large
Pena High School,[Charter],962,585858,609.0,83.839917,94.594595,84.044699,95.945946,95.270270,$601-620,Small


In [203]:
academy_budget_df= academy_adjusted_df.groupby(["Budget Bin"]).mean()
academy_budget_df = academy_budget_df[['Average Math Score %',"Passing Math %","Average Reading Score %","Passing Reading %","Overall Passing %"]]
academy_budget_df

,Average Math Score %,Passing Math %,Average Reading Score %,Passing Reading %,Overall Passing %
Budget Bin,,,,,
<$600,83.436210,93.541501,83.892196,96.459627,95.000564
$601-620,83.839917,94.594595,84.044699,95.945946,95.270270
$621-640,79.474551,77.139934,82.120471,87.468080,82.304007
>$641,77.023555,66.701010,80.957446,80.675217,73.688113


In [218]:
academy_adjusted_df.groupby('School Type')['School Size'].value_counts()

School Type   School Size
['Charter']   Small          7
              Medium         1
['District']  Medium         4
              Large          3
Name: School Size, dtype: int64

# Scores by School Size

In [204]:
academy_size_df= academy_adjusted_df.groupby(["School Size"]).mean()
academy_size_df = academy_size_df[['Average Math Score %',"Passing Math %","Average Reading Score %","Passing Reading %","Overall Passing %"]]
academy_size_df

,Average Math Score %,Passing Math %,Average Reading Score %,Passing Reading %,Overall Passing %
School Size,,,,,
Small,83.502373,93.585560,83.883125,96.593182,95.089371
Medium,78.112137,72.043261,81.564235,83.622873,77.833067
Large,77.136883,66.496861,80.978256,81.339570,73.918215


# Scores by School Type

In [212]:
academy_adjusted_df['School Type']= academy_adjusted_df['School Type'].astype('str') 
academy_type_df= academy_adjusted_df.groupby(["School Type"]).mean()
academy_type_df = academy_type_df[['Average Math Score %',"Passing Math %","Average Reading Score %","Passing Reading %","Overall Passing %"]]
academy_type_df

,Average Math Score %,Passing Math %,Average Reading Score %,Passing Reading %,Overall Passing %
School Type,,,,,
['Charter'],83.473852,93.620830,83.896421,96.586489,95.103660
['District'],76.956733,66.548453,80.966636,80.799062,73.673757
